# Spooky Author Identification (pt-BR)
This is an instructional notebook for Brazilian Portuguese Speakers

Este notebook é um exemplo de submissão para a competição Spooky Author Identification do Kaggle. Ele foi pensado com o objetivo de que iniciantes no ramo de Machine Learning tenham um guia simples de como fazer sua primeira submissão.

O que estamos usando aqui é um classificador do tipo Naive Bayes e usando o básico dos pacotes d sicikit learn para trabalhar com texto.

Vamos iniciar importando as bibliotecas básicas

In [ ]:
import pandas as pd   # manipulacao de dados do CSV
import numpy as np    # algebra linear e calculos em geral

Nosso próximo passo é carregar os arquivos usando pandas.

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
submission_df = pd.read_csv('../input/sample_submission.csv')

Vamos dar uma olhada nesses arquivos? O primeiro ponto importante é sabermos que tipo de informação nós temos.

In [ ]:
train_df.head()

Legal. O arquivo test_df tem o mesmo format, exceto que não tem a coluna de author. É o que estamos querendo prever, certo?

In [ ]:
test_df.head()

O arquivo de submission é um exemplo de como devemos mandas as previsões. Vamos usar ele como template mais pra frente. O importante aqui é notar que na submissão, a previsão é baseada em probabilidades.

In [ ]:
submission_df.head()

Mais um passo importante na hora de entendermos os dados é saber se o nosso dataset está equilibrado. Isso pode alterar o nosso tipo de abordagem de classificação.

In [ ]:
count_by_author = train_df.groupby('author')['id'].count()
count_by_author

Ok. Ele não é perfeitamente equilibrado, mas também não é muito ruim. Vamos calcular o nosso baseline, ou seja: qual seria a performance de um classificador tirivial, que sempre prediz a classe mais provável? (Nesse caso, Edgard Allan Poe)

In [ ]:
count_by_author.max()/count_by_author.sum()

Em resumo: nosso classificador "inteligente" tem que acertar mais de 40% pra ser melhor que o classificador trivial.

## Preparação dos Dados

O sklearn, que é a biblioteca de ML que vamos usar, trabalha apenas com dados numéricos. Logo, vamos ter que dar uma massageada nos dados, porque tudo o que temos no CSV é texto. Lembra o que vimos lá em cima?

### Bag of Words
O primeiro passo que vamos fazer é transformar o texto de cada linha em uma representação de Bag of Words. Isso vai incluir o processo de tokenização.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()

Vamos pedir para que o CountVectorizer "aprenda" o vocabulário do nosso texto. Isso vai fazer com que ele seja capaz de trabalhar com Bag of Words.

*Importante*: veja que estamos ensinando o vocabulário usando os dois datasets: train e test! Isso é importante porque pode ser que existam palavras no dataset de test que não exista no dataset de treino.

In [ ]:
bow.fit(test_df.append(train_df, sort=False)['text'])

Ótimo! Agora ele conhece o vocabulário. Quer ver?

Vamos começar espiando o testo da primeira amostra do dataset de treino.

In [ ]:
print(train_df.iloc[0].text)

In [ ]:
print(bow.vocabulary_['process'])
print(bow.vocabulary_['afforded'])
print(bow.vocabulary_['ascertaining'])
print(bow.vocabulary_['this'])

In [ ]:
print("Tamanho do Vocabulario aprendido:", len(bow.vocabulary_))

Nesse processo as palavras foram todas convertidas para minusculas

In [ ]:
'This' in bow.vocabulary_

Ótimo. Agora que já temos um vocabulário, hora de transformar o nosso texto em números, usando a representação de Bag of Words. O resultado vai ser uma matriz em que cada linha é uma amostra (alinhada com o dataset de treinamento) e cada coluna representa o número de vezes que aquela palavra apareceu.

In [ ]:
train_X_bow = bow.transform(train_df['text'])
train_X_bow

É um pouco difícil de trabalhar com matrizes esparsas. Sabemos que essa matriz tem 19579 linhas (são as amostras) e 28300 colunas (são as palavras). Vamos espiar uma linha pra entender melhor o que está acontecendo? Vamos espiar a primeira linha.

In [ ]:
x_bow_0 = train_X_bow[0].toarray().reshape(-1)
x_bow_0

Ainda difícil de ver. Vamos reordenar, pra ficar mais fácil. Vamos ordenar em ordem decrescente, para saber quais as palavras mais frequentes nesse texto. Pelo modelo de bag of words, essas seriam as mais importantes.

In [ ]:
idx = np.argsort(-x_bow_0)[:20]
print(idx)
print(x_bow_0[idx])
print(np.array(bow.get_feature_names())[idx])

Fica aí a reflexão: você acha que essa ordenação de fato reflete a importância dessas palavras? Tem jeito de melhorar isso?

Dica: stop words e TF-IDF

### TF-IDF
TF-IDF para os íntimos, é a sigla de Term Frequency - Inverse Document Frequency. Trata-se de uma transformação sobre o modelo de Bag of Words para tentar resolver alguns dos problemas que vimos ali em cima.

Essa transformação faz uma mágica: ele diminui a importância de palavras que aparecem em muitos documentos (como the, of, etc) e aumenta a importância de palavras que são mais raras naquelo documento: ou seja - que provavelmente são mais alinhadas com o estilo do autor ou do assunto. Quer saber como calcular o TF-IDF? É fácil, mas a gente não vai tratar disso aqui. [https://en.wikipedia.org/wiki/Tf%E2%80%93idf]

Primeiro nós precisamos fazer o TF-IDF Transformer "entender" quais as palavras comuns e quais não são comuns.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tfidf.fit(train_X_bow)

Agora que ele já sabe quais as palavras comuns, hora de transformar o nosso bag of words

In [ ]:
train_X_tfidf = tfidf.transform(train_X_bow)
train_X_tfidf

Note que o tamanho da matriz é exatamente o mesmo. Vamos espiar o conteúdo?

In [ ]:
x_tfidf_0 = train_X_tfidf[0].toarray().reshape(-1)
x_tfidf_0

Ainda difícil de ver porque é bem esparso. Vamos ordenar?

In [ ]:
idx = np.argsort(-x_tfidf_0)[:20]
print(idx)
print(x_tfidf_0[idx])
print(np.array(bow.get_feature_names())[idx])

E aí? Agora faz mais sentido esse critério de importância (ou característica) de cada autor?

Fica aqui mais uma reflexão: é sempre desejável termos essa representação

### LabelEncoding do target
O nome do autor também é texto. Vamos ter que dar um jeito de converter os nomes dos autores para valores numérico, porque é o jeito que o sklearn trabalhar. Os 3 textos que temos que transformar são EAP, HPL e MWS. 

O método que vamos usar é chamado de Label Encoding. Ou seja: a cada texto, vamos atribuir um inteiro. Como por exemplo: EAP = 0, HPL = 1 e MWS = 2.

Antes de atribuirmos, precisamos que o Encoder "aprenda" quais as categorias existentes.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_df['author'])
le.classes_

Ótimo! Agora o LabelEncoder já sabe quais são as categorias que ele tem que mapear. Agora precisamos efetivamente converter a coluna author. Bora lá.

In [ ]:
train_y = le.transform(train_df['author'])
train_y

## Treinando um Modelo Preditivo
Agora que já temos os dados preparados e transformados em dados numéricos, podemos treinar o nosso modelo de machine learning.

Para essa competição, vamos usar um classificador que em geral tem uma boa performance trabalhando com texto. Ele é baseado em estatística bayesiana e é normalmente chamado de Naive Bayes (Naive porque ele acredita que os dados não tenham relação entre si...). Não vamos entrar em detalhes aqui do que é um modelo bayesiano. O que importrante pra gente: ele é bom em calcular probabilidades. Se ele sabe que 40% dos textos são do Edgard Alan Poe e que a palavra "ascertaining" tem 0,03% de chance de aparecer num texto do Poe e que "uniform" tem 0,01% de chance de aparecer num texto do Poe, um texto que tem as palavras "uniform" e "ascertaining" tem qual a probabilidade de ser um texto do Poe? E da Mary Shelley? E do Lovecraft?

Esse é o tipo de cálculo que esse classificador faz. Quer ver mais detalhes? https://en.wikipedia.org/wiki/Naive_Bayes_classifier

Curiosidade: os filtros de Spam funcionam exatamente com esse tipo de classificador.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=1.0)

Para treinar o modelo, temos que passar pra ele as "features" ou características - nesse caso, TFIDF - e quais são os targets, pra que ele possa calcular as probabilidades.

In [ ]:
nb.fit(train_X_tfidf, train_y)

Modelo treinado. Com o modelo treinado, podemos começar usá-lo para fazer predições. Só de farra, vamos usar o próprio dataset de treinamento para fazer uma previsão e ver como ele se comporta.

In [ ]:
y_pred = nb.predict(train_X_tfidf)
y_pred

Ou seja: ele previu que o primeiro texto é do Poe (0), o segundo também, ... e o último é do Lovecraft (1)

## Avaliando o Modelo
Mas e aí. Esse modelo é bom? Qual é a taxa de acerto?

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(train_y, y_pred)

Boa! 89%! Nada mal! ;-) Bem melhor que os 40% do nosso classificador trivial.

Ponto de atenção: veja que nós estamos usando os dados de treino pra prever os dados de treino. No mundo real não é assim que funciona: o seu modelo tem que prever dados que nunca viu na vida. Logo, não é um número que dá pra confiar. Vamos ver isso um pouco mais pra frente.

No comecinho desse notebok nós falamos que o que a competição pede na verdade é qual é a probabilidade de autoria. Veja um exemplo do arquivo de submissão.

In [ ]:
submission_df.head()

Nós conseguimos gerar essas probabilidades usando o método predict_proba.

In [ ]:
y_pred_proba = nb.predict_proba(train_X_tfidf)
y_pred_proba

Nessa competição, a métrica que eles escolheram não foi accuracy.

É uma outra métrica chamada Log Loss, ou Cross Entropy. O que essa métrica faz (através de uma fórmula não muito complicada) é penalizar quando você calcula as probabilidades de forma muito errada e te premia quando as probabilidades estão certas. 

Ou seja: se você previou com 100% de certeza que o texto é do Edgard Alan Poe mas errou, ele te dá uma penalização alta. Se você previou com 40% e errou, ele ainda te penaliza, mas a penalização é menor.

Se quiser saber como calular, pode dar uma espiada aqui: https://en.wikipedia.org/wiki/Cross_entropy

*Importante*: ao contrário da Acurácia, o Log Loss melhora quando diminui. Quando menor, melhor.

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_pred=y_pred_proba, y_true=train_y)

E isso aí? É bom ou ruim? Log Loss é uma métrica difícil de interpretar. O que posso dizer é que nesse momento, com esse Log Loss você estaria mais ou menos na metade do Leader Board da competição. Mas lembre-se: não dá pra confiar, porque estamos avaliando o modelo com os próprios dados que usamos pra treinar.

### Cross Validation
Para poder avaliar o modelo mais próximo do mundo real, nós temos que fazer previsões com dados que não foram vistos durante o treino. Isso normalmente é feito separando os dados, reservando parte apenas para avaliar o modelo. Essa técnica é chamada de Holdout ou Split e funciona bem quando você tem muitos dados.

O que nós vamos usar aqui é um método diferente, chamado de K-Fold cross validation. É parecido com o holdout, só o que nós vamos fazer é separar o dataset em k grupos diferentes. E aí vamos usar o primeiro grupo como holdout e os seguintes para treinar. Em seguida pegamos o segundo como holdout e os outros pra treinar. E assim por diante. Na prática, se temos um 10-Fold cross validation, vamos treinar o modelo 10 vezes e avaliar 10 vezes usando dados que não foram usados durante o treinamento.

Vamos começar criando a classe que "splita" o nosso DataFrame

In [ ]:
from sklearn.model_selection import cross_val_predict, StratifiedKFold
cv = StratifiedKFold(n_splits=10, random_state=42)

O próximo passo é usar o cross_val_predict (dá pra usar o cross_val_score, também). O que ele faz é fazer fit e predict nas 10 folds e gerar o que a comunidade chama de "Out of Fold Prediction". No final, calculamos a acurácia.

In [ ]:
y_pred = cross_val_predict(MultinomialNB(alpha=1.0),
                           train_X_tfidf,
                           train_y,
                           cv=cv)
accuracy_score(train_y, y_pred)

Notou que o valor é menor do que quando calculamos a acurácia apenas usando o dataset de treinamento? Esse número provavelmente é muito mais próximo do desempenho que ele vai ter na vida real.

Vamos fazer o mesmo com o log_loss, que é a métrica oficial dessa competição.

In [ ]:
y_pred_proba = cross_val_predict(MultinomialNB(alpha=1.0),
                                 train_X_tfidf,
                                 train_y,
                                 cv=cv,
                                 method='predict_proba')
from sklearn.metrics import log_loss
log_loss(train_y, y_pred_proba)

Notou que ele é pior que o train score?

## Preparando a Submissão

Hora de preparar a nossa submissão. Vamos pegar o nosso modelo e prever os resultados a partir do arquivo de teste. É assim que o Kaggle avalia o seu modelo: entendendo como ele se comporta em um conjunto de dados que não foi visto durante o treinamento. Antes de fazer a previsão, precisamos aplicar exatamente as mesmas transformações que fizemos no dataset de treinamento.

Começando com a conversão pra bag of words...

In [ ]:
test_X_bow = bow.transform(test_df['text'])
test_X_bow

Aplicando o TF-IDF...

In [ ]:
test_X_tfidf = tfidf.transform(test_X_bow)
test_X_tfidf

E, finalmente, fazendo a previsão. Vamos usar o predict_proba, porque é o que o Kaggle espera.

In [ ]:
y_pred_proba = nb.predict_proba(test_X_tfidf)
y_pred_proba

### Gerando o arquivo de submissão

Agora só falta gerar o arquivo que vai ser submetido. Lembrando, o formato de arquivo esperado é o seguinte:

In [ ]:
submission_df.head()

Agora precisamos atribuir os valores que foram previstos pelo modelo.

In [ ]:
submission_df['EAP'] = y_pred_proba[:, 0]
submission_df['HPL'] = y_pred_proba[:, 1]
submission_df['MWS'] = y_pred_proba[:, 2]

O resultado final vai ser parecido com isso.

In [ ]:
submission_df.head()

Ou seja: para o primeiro texto, nosso modelo acha que tem 29% de chance de ser do Poe, 8% de chance de ser do Lovecraft e 62% da Mary Shelley. E aí? Qual é o autor real? Bom, a realidade é que niguém sabe. O kaggle mantem essa informação secreta e usa esses dados secretos para calcular o score real do seu modelo.

Só por referência, esse é o texto que ele está prevendo na primeira linha. Você acha que acertamos dizendo que é da Mary Shelley? Nunca saberemos. :-)

In [ ]:
test_df.iloc[0].text

Agora só falta gravar o arquivo. E submeter.

In [ ]:
submission_df.to_csv('basic-submission-multonmial-nb2.csv', index=False)

Anote aqui, pra depois deixar documentado.

Qual foi o seu Log Loss calculado no Cross Validation?

CV=?

Qual foi o seu Los Loss calculado pelo Kaggle no Leader Board?

LB=?

## Sugestões de Exercícios
Daqui pra frente, a sugestão é tentar melhorar o modelo. Tente alterar os parâmetros de modelagem e descobrir qual o melhor modelo que você conseguir. Tente alterar o parâmetro alpha no classificador. Seus resultados melhoram? E se não usarmos Tf-Idf, usando direto o Bag of Words? E se usássemos stop words, teria diferença?

Se souber trabalhar com outros classificadores, tente mudar o tipo de classificador (Ex: RandomForest).

Use a sua Cross Validation para descobrir qual o melhor modelo e no final faça uma nova submissão. Documente os resultados.